In [4]:
import sys
import os
os.chdir(sys.path[3])

In [120]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [41]:
raw_data = pd.read_csv(r'data/process/new_data.csv', skiprows=[1, 2]).reset_index()


The questions we are going to use for the following model are stored in the cols variable. Q8 (What type of lock did you use to secure your bicycle? - Selected Choice) has been removed since it includes 951 Nun Values (that means 47% of participants didn't answer that question). The rest of the filtered questions have been removed since I thought they were irrelevant.

In [47]:

cols= ['index','Q2','Q4','Q5','Q6','Q7','Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q17']

In [33]:
# Just for presentation
pd.read_csv(r'data/process/new_data.csv', skiprows=[2]).reset_index()

,index,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,Q33_9_TEXT,Q34,Q35,Q36,Q37,Q38,Q39,Q39_10_TEXT,Q40,Q40_3_TEXT
0,0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,...,What were the main reasons you cycled more oft...,In what year were you born?,What is your gender?,What is your annual household income after taxes?,How many bicycles do you currently own?,What is the highest level of education you hav...,What do you consider your main ethnic origin o...,What do you consider your main ethnic origin o...,Where did you hear about this survey? - Select...,Where did you hear about this survey? - Other ...
1,1,4/21/2022 14:05,4/21/2022 14:10,IP Address,169.231.161.135,100,318,TRUE,4/21/2022 14:10,R_sRQ635z5e2hc1t7,...,NaN,1996,Woman,"$200,000 or more per year",One,"Graduate degree (e.g., Master’s, Professional ...","European (e.g., British Isles, German, French,...",NaN,Email,NaN
2,2,4/14/2022 17:28,4/14/2022 17:41,IP Address,128.111.106.41,22,770,FALSE,4/21/2022 17:41,R_21uo7CnrNd8VNIx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4/21/2022 19:27,4/21/2022 19:31,IP Address,70.77.83.154,100,229,TRUE,4/21/2022 19:31,R_7V5ufQv2WtbkiSR,...,NaN,1973,Man,I prefer to not answer,Three,Bachelor’s degree (university 4-year degree) ...,"European (e.g., British Isles, German, French,...",NaN,"A bicycle registry/recovery system (e.g., Bike...",NaN
4,4,4/14/2022 19:04,4/14/2022 20:18,IP Address,70.185.131.128,22,4454,FALSE,4/21/2022 20:18,R_qPdUVtdN6wMTtIt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2021,6/6/2022 22:07,6/6/2022 22:14,IP Address,47.146.75.18,100,423,TRUE,6/6/2022 22:14,R_3fYRblhWSYPWCj8,...,NaN,1970,Woman,"Between $100,000 and $149,999 per year",Three,Bachelor’s degree (university 4-year degree) ...,"European (e.g., British Isles, German, French,...",NaN,Email,NaN
2022,2022,5/24/2022 12:17,5/24/2022 12:20,IP Address,174.90.223.242,69,191,FALSE,6/7/2022 12:20,R_3O7a8xS7l3i8EZG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023,2023,6/8/2022 3:19,6/8/2022 3:30,IP Address,108.211.181.111,100,697,TRUE,6/8/2022 3:30,R_3gO85Ni57rH0pTr,...,NaN,NaN,Prefer not to say,I prefer to not answer,I prefer to not answer,"Graduate degree (e.g., Master’s, Professional ...",I prefer to not answer,NaN,Other (please specify):,acquientance
2024,2024,5/25/2022 13:39,5/25/2022 13:39,IP Address,50.205.153.166,2,12,FALSE,6/8/2022 13:39,R_1jwTkRmNyNow9Tz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
raw_features = raw_data[cols]
raw_features

,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q17
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,$250-$499,NaN,NaN,Yes
1,1,"Entire bicycle (i.e., frame and components)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,$1000-$1999,​No,BMX,No
3,3,"Entire bicycle (i.e., frame and components)",2017,May,Evening (18:00-23:59),"Yes, locked one or more wheels (i.e., bike loc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,$2000-3999,​No,Folding,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,$1000-$1999,​No,Mountain,No
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,$2000-3999,​No,Mountain,No
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,$2000-3999,​No,Road,No
2023,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The code discards participants who answer more than three null questions

In [115]:
filtered_nun = raw_features[raw_features.count(axis=1) > 3]

,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q17
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,$250-$499,NaN,NaN,Yes
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,$1000-$1999,​No,BMX,No
3,3,"Entire bicycle (i.e., frame and components)",2017,May,Evening (18:00-23:59),"Yes, locked one or more wheels (i.e., bike loc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,$2000-3999,​No,Folding,No
5,5,"Entire bicycle (i.e., frame and components)",2020,Don’t know/not sure,Evening (18:00-23:59),"No, unlocked",Outdoor bike rack,No,"The police ,A bicycle registry/recovery system...",No,$500-$999,​No,Mountain,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,"Entire bicycle (i.e., frame and components)",2022,March,Overnight (00:00-05:59),"No, unlocked","Outside in the yard or on a balcony, porch, or...",No,The police,Yes,$250-$499,​No,Mountain,Yes
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,$1000-$1999,​No,Mountain,No
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,$2000-3999,​No,Mountain,No
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,$2000-3999,​No,Road,No


In [116]:
for col in cols[1:]:
    print(filtered_nun.groupby(by = col,dropna=False).count()['index'])

Q2
A major bicycle part necessary for riding (i.e., saddle, wheel, pedal, or handlebar)      25
Entire bicycle (i.e., frame and components)                                             1826
NaN                                                                                       15
Name: index, dtype: int64
Q4
2017                    45
2018                    54
2019                   262
2020                   325
2021                   695
2022                   159
Don’t know/not sure      7
NaN                    319
Name: index, dtype: int64
Q5
April                  163
August                 145
December                85
Don’t know/not sure    131
February                88
January                 97
July                   137
June                   281
March                  116
May                    264
November                94
October                127
September              134
NaN                      4
Name: index, dtype: int64
Q6
Afternoon (12:00-17:59)    362
Don’t 

### Missing Values

First, a table is created, sorted from the question with the least NuNs to the question with the most NuNs

In [117]:
temp_dic = {'col_q':[],'nun_count':[]}
for col in cols[1:]:
    temp_groupby = filtered_nun.groupby(by = col,dropna=False).count()['index']
    temp_dic['col_q'].append(temp_groupby.index.name)
    temp_dic['nun_count'].append(temp_groupby[[np.nan]][0])
df_col_q_nun_count = DataFrame(temp_dic)
what_to_deal_first = df_col_q_nun_count.sort_values(by='nun_count')
what_to_deal_first

,col_q,nun_count
2,Q5,4
4,Q7,8
3,Q6,11
0,Q2,15
9,Q13,26
5,Q9,28
6,Q10,28
7,Q11,28
8,Q12,29
12,Q17,42


The NuNs of the question with the fewest NuNs will be replaced with the most popular answer

In [118]:
min_nun = what_to_deal_first.iloc[0]['col_q']
print("The question with the fewest NuN values is {} ".format(min_nun))
replace_nun = filtered_nun.groupby(by = min_nun ,dropna=False).count()['index'].idxmax()
print("the most popular answer for {} is {} ".format(min_nun,replace_nun))
filtered_nun[min_nun] = filtered_nun[min_nun].fillna(replace_nun)


The question with the fewest NuN values is Q5 
the most popular answer for Q5 is June  


C:\Users\Achituv\AppData\Local\Temp\ipykernel_19560\2407467839.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nun[min_nun] = filtered_nun[min_nun].fillna(replace_nun)


In [134]:
filtered_nun

,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q17
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,$250-$499,NaN,NaN,Yes
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,$1000-$1999,​No,BMX,No
3,3,"Entire bicycle (i.e., frame and components)",2017,May,Evening (18:00-23:59),"Yes, locked one or more wheels (i.e., bike loc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,$2000-3999,​No,Folding,No
5,5,"Entire bicycle (i.e., frame and components)",2020,Don’t know/not sure,Evening (18:00-23:59),"No, unlocked",Outdoor bike rack,No,"The police ,A bicycle registry/recovery system...",No,$500-$999,​No,Mountain,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,"Entire bicycle (i.e., frame and components)",2022,March,Overnight (00:00-05:59),"No, unlocked","Outside in the yard or on a balcony, porch, or...",No,The police,Yes,$250-$499,​No,Mountain,Yes
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,$1000-$1999,​No,Mountain,No
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,$2000-3999,​No,Mountain,No
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,$2000-3999,​No,Road,No


In [136]:
test  = filtered_nun.apply(lambda x: x.astype('category').cat.codes)

In [119]:
filtered_nun.groupby(by = min_nun ,dropna=False).count()['index']

Q5
April                  163
August                 145
December                85
Don’t know/not sure    131
February                88
January                 97
July                   137
June                   285
March                  116
May                    264
November                94
October                127
September              134
Name: index, dtype: int64

In [124]:
clf = RandomForestClassifier()

In [128]:
cur_field = what_to_deal_first.iloc[1]['col_q']
cur_field

'Q7'

In [ ]:
data = filtered_nun[filtered_nun[cur_field].notna()]

In [133]:
X = data[['Q5']].to_numpy()
X

array([['Don’t know/not sure'],
       ['March '],
       ['May '],
       ...,
       ['May '],
       ['Don’t know/not sure'],
       ['January ']], dtype=object)

In [ ]:
X = demographic_data[demographic_data.columns[:-1]].to_numpy()

In [ ]:
clf.fit(X, y)